In [17]:
import pandas as pd
import itertools

In [18]:
def perform_backtest(df, short_period, intermediate_period, long_period):
    # This is a very simple logic just for illustration
    # Here we check how many times short MA is above both intermediate and long MAs
    positive_crosses = df[df[f'SMA_{short_period}'] > df[f'SMA_{intermediate_period}']]
    positive_crosses = positive_crosses[positive_crosses[f'SMA_{short_period}'] > positive_crosses[f'SMA_{long_period}']]
    
    # Return the count of positive crosses as the "metric"
    # This is just a placeholder, and in a real scenario, you'd use a more complex strategy
    return len(positive_crosses)


In [24]:
# Define ranges
short_range = range(5, 51)
intermediate_range = range(51, 101)
long_range = range(101, 201)

In [20]:
best_combination = None
best_metric = -float('inf')  # Example: if the goal is to maximize some metric

In [25]:
# Get a dataframe with Close as the data and Date as the index 
import fndata
import os
from dotenv import load_dotenv
load_dotenv()
symbol = 'SCHA'
period = '10' # required parameter, not used in this example
fmpkey = os.getenv('FMPKEY')
url = f"https://financialmodelingprep.com/api/v3/technical_indicator/daily/{symbol}?period={period}&type=sma&apikey={fmpkey}"
data = fndata.get_jsonparsed_data(url)
df = pd.DataFrame(data)
df = df[['date', 'close']]
#set the date as the index
df.rename(columns={'date': 'Date', 'close': 'Close'}, inplace=True) 
df.set_index('Date', inplace=True)
df.index = pd.to_datetime(df.index)

df = df.head(200)

df

,Close
Date,
2023-09-29,41.42
2023-09-28,41.58
2023-09-27,41.17
2023-09-26,40.85
2023-09-25,41.40
...,...
2022-12-19,39.93
2022-12-16,40.50
2022-12-15,40.85


In [26]:
# Evaluate all combinations
for short, intermediate, long in itertools.product(short_range, intermediate_range, long_range):
    if short < intermediate < long:
        df[f'SMA_{short}'] = df['Close'].rolling(window=short).mean()
        df[f'SMA_{intermediate}'] = df['Close'].rolling(window=intermediate).mean()
        df[f'SMA_{long}'] = df['Close'].rolling(window=long).mean()
        
        # Perform backtesting and calculate performance metric
        metric = perform_backtest(df, short, intermediate, long)  # Define this function based on your strategy
        
        if metric > best_metric:
            best_metric = metric
            best_combination = (short, intermediate, long)

print("Best Combination of SMA Periods: ", best_combination)


Best Combination of SMA Periods:  (20, 50, 51)
